In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()


url = "https://elshennawy.com/ProductsList?categoryId=1&brandId=2"

driver.get(url)
time.sleep(5)


response = requests.get(url)


if response.status_code == 200:

    soup = BeautifulSoup(response.content, 'html.parser')


    products = soup.find_all('div', class_='prd-info')


    product_names = []
    product_brands = []
    product_storage = []
    product_colors = []
    product_prices_new = []
    product_prices_old = []
    product_savings = []
    product_ids = []


    for product in products:

        name = product.find('h2', class_='prd-title').get_text(strip=True)
        product_names.append(name)


        brand = product.find('div', class_='prd-tag').get_text(strip=True)
        product_brands.append(brand)


        storage = product.find_all('div', class_='d-block')[0].get_text(strip=True).replace("Storage ", "")
        product_storage.append(storage)


        blocks = product.find_all('div', class_='d-block')
        color = blocks[1].get_text(strip=True).replace("Color ", "") if len(blocks) > 1 else "N/A"
        product_colors.append(color)


        price_new = product.find('div', class_='price-new').get_text(strip=True)
        product_prices_new.append(price_new)


        price_old = product.find('div', class_='price-old').get_text(strip=True) if product.find('div', class_='price-old') else None
        product_prices_old.append(price_old)


        savings = product.find('div', class_='price-comment').get_text(strip=True) if product.find('div', class_='price-comment') else None
        product_savings.append(savings)

    wishlist_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.label-wishlist')
    for button in wishlist_buttons:

       onclick_attr = button.get_attribute('onclick')
       product_id = onclick_attr.split('(')[1].split(',')[0]
       product_ids.append(product_id)



    df = pd.DataFrame({
        'Product Name': product_names,
        'Brand': product_brands,
        'Storage': product_storage,
        'Color': product_colors,
        'New Price': product_prices_new,
        'Old Price': product_prices_old,
        'Savings': product_savings ,
        'id' : product_ids
    })


    print(df)


    df.to_csv('elshnawy_apple.csv', index=False)


else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

ModuleNotFoundError: No module named 'selenium'

In [ ]:



driver = webdriver.Chrome()

driver.get(url)


time.sleep(5)


wishlist_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.label-wishlist')


product_dict = {}

for button in wishlist_buttons:

    onclick_attr = button.get_attribute('onclick')
    product_id = onclick_attr.split('(')[1].split(',')[0]


    product_url = f"https://elshennawy.com/ProductsList/Show?productId={product_id}"


    product_dict[product_id] = product_url


print(product_dict)


driver.quit()


{'15768': 'https://elshennawy.com/ProductsList/Show?productId=15768', '16355': 'https://elshennawy.com/ProductsList/Show?productId=16355', '16356': 'https://elshennawy.com/ProductsList/Show?productId=16356', '16357': 'https://elshennawy.com/ProductsList/Show?productId=16357', '15960': 'https://elshennawy.com/ProductsList/Show?productId=15960', '16353': 'https://elshennawy.com/ProductsList/Show?productId=16353', '16034': 'https://elshennawy.com/ProductsList/Show?productId=16034', '16323': 'https://elshennawy.com/ProductsList/Show?productId=16323', '16461': 'https://elshennawy.com/ProductsList/Show?productId=16461', '16462': 'https://elshennawy.com/ProductsList/Show?productId=16462', '16463': 'https://elshennawy.com/ProductsList/Show?productId=16463', '15961': 'https://elshennawy.com/ProductsList/Show?productId=15961', '15962': 'https://elshennawy.com/ProductsList/Show?productId=15962', '15963': 'https://elshennawy.com/ProductsList/Show?productId=15963', '15964': 'https://elshennawy.com/

In [ ]:



all_products_df = pd.DataFrame()


for product_id, url in product_dict.items():

    response = requests.get(url)


    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')


        holder = soup.find('div', class_='holder')

        if holder:

            tables = holder.find_all('table', class_='table')


            product_data = []


            for table in tables:

                section_headers = table.find_all('thead')
                for section_header in section_headers:
                    section_title = section_header.get_text(strip=True)


                    rows = table.find_all('tr')
                    for row in rows:

                        columns = row.find_all('td')
                        if len(columns) == 2:
                            key = columns[0].get_text(strip=True)
                            value = columns[1].get_text(strip=True)
                            product_data.append([product_id, section_title, key, value])


            product_df = pd.DataFrame(product_data, columns=['Product ID', 'Section', 'Specification', 'Details'])


            all_products_df = pd.concat([all_products_df, product_df], ignore_index=True)

    else:
        print(f"Failed to retrieve the webpage for product ID {product_id}. Status code: {response.status_code}")


print(all_products_df)


all_products_df.to_csv('elshnawy_apple_specs.csv', index=False)


      Product ID  Section Specification  \
0          15768  Network    Technology   
1          15768  Network      2G bands   
2          15768  Network      3G bands   
3          15768  Network      4G bands   
4          15768  Network         Speed   
...          ...      ...           ...   
13343      10063  BATTERY           USB   
13344      10063  BATTERY       Sensors   
13345      10063  BATTERY        Others   
13346      10063  BATTERY          Type   
13347      10063  BATTERY      Charging   

                                                 Details  
0      5G bands\t1, 2, 3, 5, 7, 8, 12, 20, 25, 26, 28...  
1      GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...  
2      HSDPA 850 / 900 / 1700(AWS) / 1900 / 2100  \tC...  
3      1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 2...  
4                  HSPA, LTE-A, 5G, EV-DO Rev.A 3.1 Mbps  
...                                                  ...  
13343                                                     
13344      